# Lightweight Face Detector TF Implementation

Based almost entirely off the paper: "A lightweight face detector by integrating the convolutional neural network with the image pyramid"

Github link: https://github.com/JiapengLuo/lightweight_face_detector


In [17]:
import tensorflow as tf
print(tf.__version__)

import os
import numpy as np
import time
from PIL import Image, ImageDraw

# saved_model_path = "/Users/johngearig/Downloads/lightweight_face_detector-master/checkpoints/fast_mode_enable_hflip"
# im_path = '/Users/johngearig/Downloads/lightweight_face_detector-master/demo1.jpg'
# out_path = '/Users/johngearig/Downloads/lightweight_face_detector-master/demo1_detected.jpg'

saved_model_path = "/Users/james/Downloads/lightweight_face_detector-master/lightweight_face_detector-master/checkpoints/fast_mode_enable_hflip"
im_path = "Images/0--Parade/0_Parade_marchingband_1_5.jpg"
out_path = "Images/0--Parade/0_Parade_marchingband_1_5_Detected.jpg"

1.13.1


NOTE: This will not run on any recent tensorflow release since it relies heavily on tf.Session() which was discontinued in tf.__version__ 2.0.

The Pillow import is only for viewing images and could be swapped for another image library. 

In [18]:
def draw_boxes(img, boxes, color='green', width=2):
    '''
    draw the boxes in the img
    :param img: Pillow Image or numpy
    :param boxes: boxes, [[ymax, xmax, ymin, xmin]...]
    :param color: color
    :return: Image drawed boxes
    '''
    if isinstance(img, np.ndarray):
        img = Image.fromarray(img.astype(np.uint8), mode='RGB')
    elif not isinstance(img, Image.Image):
        raise ValueError("image must be a Image or ndarray.")
    draw = ImageDraw.Draw(img)
    for box in boxes:
        draw.rectangle([box[1], box[0], box[3], box[2]], outline=color, width=width)
    return img

In [19]:
def run():
    sess = tf.compat.v1.Session()
    tf.saved_model.load(sess, ['serve'], saved_model_path )
    
    inp = sess.graph.get_tensor_by_name('input_images:0')
    boxes_tensor = sess.graph.get_tensor_by_name('boxes:0')
    scores_tensor = sess.graph.get_tensor_by_name('scores:0')

    im = Image.open(im_path)
    im = np.array(im).astype(np.float32)
    
    start = time.time()
    boxes, scores = sess.run([boxes_tensor,scores_tensor], feed_dict={inp: np.expand_dims(im, axis=0)})
    end = time.time()
    print(end - start)
    
    """print(boxes.shape)
    print(boxes.dtype)
    print(scores.shape)
    scores = np.reshape(scores, (1,200,1))
    
    universal = np.concatenate((boxes, scores))
    print(universal.shape)"""
    
    boxes = boxes[0]
    scores = scores[0]
    mask = scores>0.5
    boxes = boxes[mask]
    scores = scores[mask]
    print('boxes')
    print(boxes)
    print('scores')
    print(scores)
    out_im = draw_boxes(im, boxes)
    out_im.save(out_path)

if __name__ =='__main__':
    run()

INFO:tensorflow:Restoring parameters from /Users/james/Downloads/lightweight_face_detector-master/lightweight_face_detector-master/checkpoints/fast_mode_enable_hflip\variables\variables
1.815000295639038
boxes
[[179.1147   495.18933  227.00143  533.06964 ]
 [224.51584  222.12839  264.8469   259.00592 ]
 [166.6549   920.58356  208.31926  958.56036 ]
 [190.55524  717.49146  223.84572  748.8699  ]
 [154.6563   783.63904  192.61717  814.4587  ]
 [207.72005   25.534302 237.98586   52.600792]
 [183.32559  963.8858   209.97064  985.2466  ]
 [190.24266  597.7865   218.52728  621.3916  ]
 [205.95287  313.8265   265.47736  366.62473 ]
 [ 98.289986 430.97546  127.819695 453.5537  ]
 [ 49.071083 744.5245    80.832855 773.2537  ]]
scores
[0.9999939  0.9999304  0.99912494 0.99272    0.971548   0.8964634
 0.84496075 0.78985494 0.7587339  0.62578005 0.5680562 ]


In [20]:
im = Image.open(out_path)  
im.show()




All code below is trying to figure out the correct reshapes needed to make compatible with other codebase




In [22]:

x = 5

boxes = np.random.rand(1,x,4)
scores = np.random.rand(1,x)

print(boxes.shape)
print(boxes.dtype)
print(scores.shape)

scores = np.reshape(scores, (1,x,1))
print(scores.shape)
universal = np.concatenate((boxes, scores),2)
print(universal.shape)

universal = np.reshape(universal, (1,1,x,5))
empty = np.zeros((1,1,x,2))
print(universal.shape)

universal = np.concatenate((empty, universal),3)
print(universal.shape)

universal

(1, 5, 4)
float64
(1, 5)
(1, 5, 1)
(1, 5, 5)
(1, 1, 5, 5)
(1, 1, 5, 7)


array([[[[0.        , 0.        , 0.82136708, 0.01504648, 0.27813835,
          0.62545221, 0.85360546],
         [0.        , 0.        , 0.4234704 , 0.61438293, 0.79574112,
          0.77141319, 0.68048199],
         [0.        , 0.        , 0.31205906, 0.7244687 , 0.93054864,
          0.98969232, 0.2193512 ],
         [0.        , 0.        , 0.41939425, 0.65155539, 0.91466768,
          0.65834126, 0.16409021],
         [0.        , 0.        , 0.30136986, 0.48713553, 0.39105764,
          0.25110057, 0.45292768]]]])